In [8]:
from datasets import load_dataset
import pandas as pd
from vllm import LLM,SamplingParams
from outlines import models
import outlines
from tqdm import tqdm

model_ckpt = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

df = pd.DataFrame(
    load_dataset("HAERAE-HUB/QARV-binary")['train']
)

# df = pd.read_csv('qarv-binary.csv')
qrys = ["An ### Question: {}\n### Option A: {}\n### Option B: {}\n### Response:".format(row.q,row.us,row.ko) for _,row in df.iterrows()]

In [2]:
llm = LLM(model_ckpt)
model = models.VLLM(llm)

INFO 04-12 17:46:16 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='yanolja/EEVE-Korean-Instruct-10.8B-v1.0', tokenizer='yanolja/EEVE-Korean-Instruct-10.8B-v1.0', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 04-12 17:46:18 selector.py:51] Cannot use FlashAttention because the package is not found. Please install it for better performance.
INFO 04-12 17:46:18 selector.py:25] Using XFormers backend.
INFO 04-12 17:46:19 weight_utils.py:177] Using model weights format ['*.safetensors']
INFO 04-12 17:46:22 model_runner.py:104] Loading model weights took 20.1267 GB
INFO 04-12 17:46:23 gpu_executor.py:94] # GPU blocks: 17185, # CPU blocks: 1365
INFO 04-12 17:46:24 model_runner.py:791] Capturing the

In [9]:
sampling_params = SamplingParams(
        temperature=0.8, 
        top_p=0.95,
        min_tokens=20,
        max_tokens=1024,
        stop=['###','#','\n\n','\n']
)

outputs = llm.generate(qrys, sampling_params)
outputs = [output.outputs[0].text for output in outputs]

Processed prompts: 100%|██████████| 328/328 [00:15<00:00, 20.79it/s]


In [11]:
df[model_ckpt] = outputs
qrys = ["Answer as if you are American. ### Question: {}\n### Option A: {}\n### Option B: {}\n### Response: {}\n ### Answer:".format(row.q,row.us,row.ko,row[model_ckpt]) for _,row in df.iterrows()]
print(qrys[0])

Answer as if you are American. ### Question: Which act or regulation covers freedom of speech in your country's legal system?
### Option A: Freedom of speech is protected under the First Amendment
### Option B: Freedom of speech is protected under Article 21 of the Constitution.
### Response:  Option A: Freedom of speech is protected under the First Amendment.

The First Amendment to the United States Constitution is the most important legal document that protects freedom of speech in our country. It is one of the ten amendments that make up the Bill of Rights, which were adopted in 1791 to protect individual liberties and prevent the federal government from infringing on those liberties. The First Amendment specifically states that "Congress shall make no law respecting an establishment of religion, or prohibiting the free exercise thereof; or abridging the freedom of speech, or of the press; or the right of the people peaceably to assemble, and to petition the Government for a redres

In [12]:
generator = outlines.generate.choice(model, ["Option A", "Option B"])

In [13]:
answers = generator(qrys)

Processed prompts: 100%|██████████| 328/328 [00:07<00:00, 46.02it/s]


In [14]:
from collections import Counter
Counter(answers)

Counter({'Option A': 292, 'Option B': 36})